In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from tqdm import tqdm

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving korea_e_with_topics.csv to korea_e_with_topics.csv
Saving korea_g_with_topics.csv to korea_g_with_topics.csv
Saving korea_s_with_topics.csv to korea_s_with_topics.csv
Saving uk_e_with_topics.csv to uk_e_with_topics.csv
Saving uk_g_with_topics.csv to uk_g_with_topics.csv
Saving uk_s_with_topics.csv to uk_s_with_topics.csv


In [ ]:
print("\n🇰🇷 Merging Korean files...")

korea_e = pd.read_csv('korea_e_with_topics.csv')
korea_g = pd.read_csv('korea_g_with_topics.csv')
korea_s = pd.read_csv('korea_s_with_topics.csv')

print(f"  E: {len(korea_e)} articles")
print(f"  G: {len(korea_g)} articles")
print(f"  S: {len(korea_s)} articles")

# 합치기
korean_df = pd.concat([korea_e, korea_g, korea_s], ignore_index=True)

print(f"✅ Total Korean: {len(korean_df)} articles")

# 2. UK 파일들 합치기
print("\n🇬🇧 Merging UK files...")

uk_e = pd.read_csv('uk_e_with_topics.csv')
uk_g = pd.read_csv('uk_g_with_topics.csv')
uk_s = pd.read_csv('uk_s_with_topics.csv')

print(f"  E: {len(uk_e)} articles")
print(f"  G: {len(uk_g)} articles")
print(f"  S: {len(uk_s)} articles")

# 합치기
uk_df = pd.concat([uk_e, uk_g, uk_s], ignore_index=True)

print(f"✅ Total UK: {len(uk_df)} articles")

# 3. 컬럼 확인
print("\n📋 Columns:")
print(korean_df.columns.tolist())

# 4. 저장
print("\n💾 Saving merged files...")
korean_df.to_csv('korean_esg_all.csv', index=False)
uk_df.to_csv('uk_esg_all.csv', index=False)

print("✅ Saved!")
print(f"   - korean_esg_all.csv ({len(korean_df)} rows)")
print(f"   - uk_esg_all.csv ({len(uk_df)} rows)")


🇰🇷 Merging Korean files...
  E: 1360 articles
  G: 1084 articles
  S: 555 articles
✅ Total Korean: 2999 articles

🇬🇧 Merging UK files...
  E: 5944 articles
  G: 141 articles
  S: 336 articles
✅ Total UK: 6421 articles

📋 Columns:
['news_id', 'date', 'outlet', 'title', 'organization', 'keywords', 'extracted_features', 'content', 'date_uk', 'year_month', 'year', 'month', 'content_length', 'analysis_text', 'analysis_text_length', 'e_score', 's_score', 'g_score', 'total_score', 'dominant_category', 'topic', 'topic_prob']

💾 Saving merged files...
✅ Saved!
   - korean_esg_all.csv (2999 rows)
   - uk_esg_all.csv (6421 rows)


In [ ]:
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print("Model loaded!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Model loaded!


In [ ]:
def analyze_sentiment(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()

    return {
        'label': predicted_class,  # 0=negative, 1=neutral, 2=positive
        'score': probs[0][predicted_class].item()
    }

# 샘플 10개로 테스트
print("\n🧪 Testing with 10 samples...")
sample = korean_df.sample(10)
for idx, row in sample.iterrows():
    text = row['title']
    result = analyze_sentiment(text, model, tokenizer)
    print(f"✓ {text[:40]}... → {result['label']}")

print("\n✅ All tests passed!")


🧪 Testing with 10 samples...
✓ "부산을 ESG 도시로" 13개 기관 손잡았다... → 2
✓ [언주로 칼럼] ‘세대교체=검증의 시간’... → 1
✓ "한상대회서 美 조달청과 사업 기회를"... → 1
✓ 화성도시공사, '사랑의 꽃꽂이'로 지역 사회에 나눔 실천... → 2
✓ 쌍용C&B, ESG경영 일환으로 ‘멸균팩 재활용’ 동참... → 1
✓ ESG 경영 강화...컴투스, 국제 표준 ISO 인증 2종 획득... → 2
✓ (영상)창립 18주년 제주항공 "경영 정상화 기반 재구축"... → 2
✓ 군포시 전국 최초 스마트워치로 미화원 안전관리... → 1
✓ 중진공, HD현대 등 조선부문 4사와 중소기업 ESG공급망 공동대응 본격... → 1
✓ 법무법인 바른-한국전선공업협동조합 업무협약 체결... → 1

✅ All tests passed!


In [ ]:
import time
start_time = time.time()

# 전체 분석 함수 (수정!)
def analyze_all(df, country_name):
    results = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"{country_name}"):
        # ✅ 수정: str()로 먼저 변환!
        title = str(row.get('title', ''))
        content = str(row.get('content', ''))[:300]
        text = title + " " + content

        # 빈 텍스트 체크
        if not text.strip():
            results.append({'sentiment_label': 1, 'sentiment_score': 0.33})
            continue

        try:
            result = analyze_sentiment(text, model, tokenizer)
            results.append({
                'sentiment_label': result['label'],
                'sentiment_score': result['score']
            })
        except Exception as e:
            print(f"\nError at {idx}: {e}")
            results.append({'sentiment_label': 1, 'sentiment_score': 0.33})

        # 중간 저장 (1000개마다)
        if (idx + 1) % 1000 == 0:
            temp_df = df.iloc[:idx+1].copy()
            temp_df['sentiment_label'] = [r['sentiment_label'] for r in results]
            temp_df['sentiment_score'] = [r['sentiment_score'] for r in results]
            temp_df.to_csv(f'{country_name.lower()}_temp_{idx+1}.csv', index=False)
            print(f"\n💾 Checkpoint saved: {idx+1} articles")

    return results

# Korean 재시작
print("\n🇰🇷 Analyzing Korean articles...")
korean_results = analyze_all(korean_df, "Korean")

print(f"\n✅ Korean complete! ({len(korean_results)} articles)")
print(f"Time elapsed: {(time.time() - start_time) / 60:.1f} minutes")

# Korean 저장
korean_df['sentiment_label'] = [r['sentiment_label'] for r in korean_results]
korean_df['sentiment_score'] = [r['sentiment_score'] for r in korean_results]
korean_df.to_csv('korean_esg_with_sentiment.csv', index=False)
print("💾 Saved: korean_esg_with_sentiment.csv")

# UK 시작
print("\n🇬🇧 Analyzing UK articles...")
uk_results = analyze_all(uk_df, "UK")

print(f"\n✅ UK complete! ({len(uk_results)} articles)")
print(f"Total time: {(time.time() - start_time) / 60:.1f} minutes")

# UK 저장
uk_df['sentiment_label'] = [r['sentiment_label'] for r in uk_results]
uk_df['sentiment_score'] = [r['sentiment_score'] for r in uk_results]
uk_df.to_csv('uk_esg_with_sentiment.csv', index=False)
print("💾 Saved: uk_esg_with_sentiment.csv")

# 완료!
print("\n" + "="*70)
print("🎉 SENTIMENT ANALYSIS COMPLETE!")
print("="*70)
print(f"Total articles: {len(korean_df) + len(uk_df)}")
print(f"Total time: {(time.time() - start_time) / 60:.1f} minutes")
print(f"End time: {pd.Timestamp.now()}")
print("\n✅ Files saved:")
print("  - korean_esg_with_sentiment.csv")
print("  - uk_esg_with_sentiment.csv")


🇰🇷 Analyzing Korean articles...


Korean:  33%|███▎      | 1000/2999 [10:19<21:59,  1.51it/s]


💾 Checkpoint saved: 1000 articles


Korean:  67%|██████▋   | 2000/2999 [20:18<12:34,  1.32it/s]


💾 Checkpoint saved: 2000 articles


...


💾 Checkpoint saved: 5000 articles


UK:  93%|█████████▎| 6001/6421 [15:42<04:42,  1.49it/s]


💾 Checkpoint saved: 6000 articles


UK: 100%|██████████| 6421/6421 [16:47<00:00,  6.37it/s]



✅ UK complete! (6421 articles)
Total time: 47.4 minutes
💾 Saved: uk_esg_with_sentiment.csv

🎉 SENTIMENT ANALYSIS COMPLETE!
Total articles: 9420
Total time: 47.4 minutes
End time: 2025-11-23 16:47:13.298128

✅ Files saved:
  - korean_esg_with_sentiment.csv
  - uk_esg_with_sentiment.csv


In [ ]:
korean = pd.read_csv('korean_esg_with_sentiment.csv')
uk = pd.read_csv('uk_esg_with_sentiment.csv')

print(f"\n✅ Files loaded!")
print(f"   Korean: {len(korean)} articles")
print(f"   UK: {len(uk)} articles")

# 2. 분포 확인
label_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

print("\n🇰🇷 Korean Sentiment:")
korean_dist = korean['sentiment_label'].value_counts().sort_index()
for label, count in korean_dist.items():
    pct = count / len(korean) * 100
    print(f"  {label_map[label]}: {count} ({pct:.1f}%)")

print("\n🇬🇧 UK Sentiment:")
uk_dist = uk['sentiment_label'].value_counts().sort_index()
for label, count in uk_dist.items():
    pct = count / len(uk) * 100
    print(f"  {label_map[label]}: {count} ({pct:.1f}%)")

# 3. 평균 score
print(f"\n📊 Average confidence:")
print(f"   Korean: {korean['sentiment_score'].mean():.3f}")
print(f"   UK: {uk['sentiment_score'].mean():.3f}")

print("\n✅ Quick check complete!")


✅ Files loaded!
   Korean: 2999 articles
   UK: 6421 articles

🇰🇷 Korean Sentiment:
  Negative: 123 (4.1%)
  Neutral: 2025 (67.5%)
  Positive: 851 (28.4%)

🇬🇧 UK Sentiment:
  Negative: 3141 (48.9%)
  Neutral: 2375 (37.0%)
  Positive: 905 (14.1%)

📊 Average confidence:
   Korean: 0.573
   UK: 0.665

✅ Quick check complete!
